In [123]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize

In [91]:
"""
Dataframe loaded from kaggle
https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201612.csv
"""
df = pd.read_csv('datasets/ks-projects-201612.csv', encoding='cp1252')

C:\Users\evger\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
print(df.dtypes)

ID                  int64
name               object
category           object
main_category      object
currency           object
deadline           object
goal               object
launched           object
pledged            object
state              object
backers            object
country            object
usd pledged        object
Unnamed: 13        object
Unnamed: 14        object
Unnamed: 15        object
Unnamed: 16       float64
dtype: object


In [93]:
# Fix namings
df.columns = df.columns.to_series().apply(lambda x: x.strip())
pf = df[['goal', 'main_category', 'pledged', 'backers', 'usd pledged', 'state']]

# Get rid of cancelled
pf = pf.loc[pf['state'] != 'canceled']
pf1 = df[['goal', 'main_category', 'pledged', 'backers', 'usd pledged', 'state']]

# Clear shit
pf[['goal', 'pledged', 'backers', 'usd pledged']] = pf[['goal', 'pledged', 'backers', 'usd pledged']].apply(pd.to_numeric, errors='coerce')
pf = pf.dropna()

pf1[['goal', 'pledged', 'backers', 'usd pledged']] = pf1[['goal', 'pledged', 'backers', 'usd pledged']].apply(pd.to_numeric, errors='coerce')
pf1 = pf1.dropna()

# Find dummies and add them
dummy = pd.get_dummies(pf['main_category'])
pf = pd.concat([pf, dummy], axis=1)
pf = pf.drop(['main_category'], axis=1)

dummy = pd.get_dummies(pf1['main_category'])
pf1 = pd.concat([pf1, dummy], axis=1)
pf1 = pf1.drop(['main_category'], axis=1)

# Split data
# y = pf[['state']]
# y = pd.get_dummies(y[['state']])

# y = pd.Categorical(pf[['state']])
pf["state"] = pf["state"].astype('category')
y = pf[['state']]
print(y.dtypes)
x = pf.drop(['state'], axis=1)

pf1["state"] = pf1["state"].astype('category')
y1 = pf[['state']]
x1 = pf.drop(['state'], axis=1)

C:\Users\evger\Anaconda3\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


state    category
dtype: object


In [111]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.3)

In [94]:
model = LogisticRegression()
model.fit(x_train, y_train)
y_predicted = model.predict(x_test)
accuracy = accuracy_score(y_test, y_predicted)

C:\Users\evger\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [99]:
model1 = LogisticRegression()
model1.fit(x_train1, y_train1)
y_predicted1 = model.predict(x_test1)
accuracy1 = accuracy_score(y_test1, y_predicted1)

C:\Users\evger\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [128]:
y_list = label_binarize(y_test['state'].tolist(), classes=['failed', 'live', 'successful', 'suspended'])
y_list1 = label_binarize(y_test1['state'].tolist(), classes=['failed', 'live', 'successful', 'suspended', 'canceled'])

y1 = label_binarize(y_predicted.tolist(), classes=['failed', 'live', 'successful', 'suspended'])
y2 = label_binarize(y_predicted1.tolist(), classes=['failed', 'live', 'successful', 'suspended', 'canceled'])


average_precision = average_precision_score(y_list, y1)
average_precision1 = average_precision_score(y_list1, y2)
print(average_precision, average_precision1)

0.25009427046353894 nan


C:\Users\evger\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:444: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [127]:
print(accuracy, accuracy1)

0.9621017665710403 0.963007700437868
